<a href="https://colab.research.google.com/github/TiagoIesbick/dashboard-etl/blob/main/budget_forecast_dev.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [116]:
import logging

# Set Prophet loggers to WARNING level
logging.getLogger('prophet').setLevel(logging.WARNING)
logging.getLogger('cmdstanpy').setLevel(logging.WARNING)

import pandas as pd
import numpy as np
from prophet import Prophet
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.linear_model import LinearRegression

In [117]:
# cleaning budget unit 7001
def clean_7001(df: pd.DataFrame) -> pd.DataFrame:
  df.loc[df['Proj/Ativ'] == 2870, 'Proj/Ativ'] = 4396
  df.loc[df['Proj/Ativ'].isin([2872, 1507]), ['Proj/Ativ', 'Elemento']] = 4471, 339040
  df.loc[df['Proj/Ativ'].isin([2873, 2532]), 'Proj/Ativ'] = 4413
  df.loc[df['Proj/Ativ'].isin([1505, 1503, 1373, 1506]), 'Proj/Ativ'] = 2529
  df.loc[(df['Proj/Ativ'] == 2681) & (df['Elemento'] == 319192), 'Elemento'] = 319113
  df.loc[(df['Proj/Ativ'] == 2529) & (df['Elemento'] == 449092), 'Elemento'] = 449051
  df.loc[(df['Proj/Ativ'] == 9071) & (df['Elemento'] == 319091), 'Elemento'] = 339091
  df.loc[(df['Proj/Ativ'] == 9071) & (df['Elemento'].isin([339092, 339147])), 'Elemento'] = 339047
  df.loc[~((df['Proj/Ativ'] == 9071) & (df['Vinc. Orçam.'] == 1)), 'Vinc. Orçam.'] = 6069
  df.loc[(df['Proj/Ativ'] == 2529) & (df['Elemento'].isin([319011, 319016, 319092, 319094, 339036, 339046, 339049])), 'Proj/Ativ'] = 4396
  df.loc[(df['Proj/Ativ'] == 2529) & (df['Elemento'] == 319013), 'Proj/Ativ'] = 2680
  return df


# changing the elements 339001, 339003, 339091, 339092, 332001
def change_elements(df: pd.DataFrame) -> pd.DataFrame:
  df.loc[df['Elemento'] == 339001, 'Elemento'] = 319001
  df.loc[df['Elemento'] == 339003, 'Elemento'] = 319003
  df.loc[df['Elemento'] == 339091, 'Elemento'] = 319091
  df.loc[(df['Elemento'] == 339092) & (~df['Proj/Ativ'].isin([9075, 9077])), 'Elemento'] = 319092
  df.loc[df['Elemento'] == 332001, 'Elemento'] = 339086
  return df


# filling empty cells after the first filled cell in a column with 0
def fill_zero(df: pd.DataFrame) -> pd.DataFrame:
    for col in df.columns.difference(['T', 'Comp.pagto.']):
        first_valid = df[col].first_valid_index()
        if first_valid is not None:
            df.loc[first_valid:, col] = df.loc[first_valid:, col].fillna(0)
    return df

# creating moving average dataframes
def moving_averages(df: pd.DataFrame, window: int) -> pd.DataFrame:
    df_ma = df.copy()
    df_ma.loc[:, df_ma.columns.difference(['T', 'Comp.pagto.'])] = df_ma.loc[:, df_ma.columns.difference(['T', 'Comp.pagto.'])].rolling(window).mean()
    df_ma.dropna(axis=1, how='all', inplace=True)
    return df_ma

# building prophet model
def build_prophet_model() -> Prophet:
  model = Prophet(
      # growth='logistic',
      yearly_seasonality=False,
      weekly_seasonality=False,
      daily_seasonality=False
  )
  model.add_seasonality(name='yearly', period=365.25, fourier_order=10)
  model.add_seasonality(name='monthly', period=30.5, fourier_order=5)
  return model

# normalizes and calculates the score
def normalizes_calculates_score(df: pd.DataFrame) -> pd.DataFrame:
  scaler = MinMaxScaler()
  df['r2_norm'] = scaler.fit_transform(df[['R²']])
  df['rmse_norm'] = 1 - scaler.fit_transform(df[['RMSE']])
  df['mae_norm'] = 1 - scaler.fit_transform(df[['MAE']])

  df['score'] = (
      0.5 * df['r2_norm'] +
      0.25 * df['rmse_norm'] +
      0.25 * df['mae_norm']
  )
  return df


# checks if there's any negative forecast
# checks they don't jump by more than 30% (increase or decrease), while skipping division when the previous value is 0
def check_negative_and_jump(forecast_years: dict[int, float]) -> bool:
  values = list(forecast_years.values())
  has_negative = any(v < 0 for v in values)
  has_large_jump = any(
      values[i-1] != 0 and abs((values[i] / values[i-1]) - 1) > 0.3
      for i in range(1, len(values))
  )
  return has_negative or has_large_jump


# selecting the best model
def choose_models(df: pd.DataFrame, rename_level: bool = True) -> pd.DataFrame:
  df_normalized = df.groupby('Allocation').apply(normalizes_calculates_score, include_groups=False)
  df_normalized.reset_index(inplace=True)
  chosen_models = (
      df_normalized
      .sort_values('score', ascending=False)
      .groupby('Allocation')
      .first()
  )
  if rename_level:
    chosen_models.rename(columns={'level_1': 'Type'}, inplace=True)
  chosen_models = chosen_models.loc[:, ~chosen_models.columns.map(lambda x: isinstance(x, str) and x.startswith('level'))]
  return chosen_models


# expense columns that will be readjusted due to mass segregation
mass_segregation_cols_exp = {
    '7002-2736-319003-6049', '7002-2738-319003-6049', '7002-2740-319003-6049', '7002-2742-319003-6049',
    '7002-2744-319003-6049', '7002-2747-319003-6049', '7002-2752-319003-6049', '7002-2754-319003-6049',
    '7002-2756-319003-6049', '7003-2760-319003-6050', '7003-2762-319003-6050', '7003-2764-319003-6050',
    '7003-2766-319003-6050', '7003-2768-319003-6050', '7003-2771-319003-6050', '7003-2776-319003-6050',
    '7003-2778-319003-6050', '7003-2780-319003-6050'
}


# revenue columns that will be readjusted due to mass segregation
mass_segregation_cols_rev = {
    'Contr.do Servidor Civil - Pensionistas - Plano em Capitalização - CMPA-6050', 'Contr.do Servidor Civil - Pensionistas - Plano em Capitalização - DMAE-6050',
    'Contr.do Serv. Civil-Pensionistas-Plano em Capitalização-Centralizada-6050', 'Contr.do Servidor Civil - Pensionistas - Plano em Capitalização - FASC-6050',
    'Contr.do Serv. Civil - Pensionistas - Plano em Capitalização - DEMHAB-6050', 'Contr.do Servidor Civil - Pensionistas - Plano em Capitalização - DMLU-6050',
    'Contr.do Serv.Civil -Pensionistas - Plano em Repartição - Centralizada-6049', 'Contr.do Servidor Civil - Pensionistas - Plano em Repartição - CMPA-6049',
    'Contr.do Servidor Civil - Pensionistas - Plano em Repartição - DMAE-6049', 'Contr.do Servidor Civil - Pensionistas - Plano em Repartição - DMLU-6049',
    'Contr.do Servidor Civil - Pensionistas - Plano em Repartição - DEMHAB-6049', 'Contr.do Servidor Civil - Pensionistas - Plano em Repartição - FASC-6049'
}

# def time_based_train_test_split(df: pd.DataFrame, min_len: int = 2) -> tuple[pd.DataFrame, pd.DataFrame | None]:
#     """
#     Perform a time-based train/test split.

#     Args:
#         df: DataFrame with time-ordered data.
#         min_len: Minimum length required to return a split.

#     Returns:
#         (train_df, test_df): Tuple of train and test DataFrames. Test can be None.
#     """
#     if len(df) >= 60:
#         train_df = df[:-12]
#         test_df = df[-12:]
#     elif len(df) >= 12:
#         split_idx = int(len(df) * 0.8)
#         train_df = df.iloc[:split_idx]
#         test_df = df.iloc[split_idx:]
#     elif len(df) >= min_len:
#         train_df = df
#         test_df = None
#     else:
#         return None, None

#     return train_df.reset_index(drop=True), test_df.reset_index(drop=True) if test_df is not None else None

# def str_to_float(value: str) -> float:
#   value = str(value).replace('.', '').replace(',', '.')
#   value = re.sub(r'[A-Za-z]|\s', '', value)
#   return float(0) if value == '' else float(value)

# def agrupamento_format(value: str) -> str:
#   if re.search(r'^0{2}', value[:2]):
#     value = '   ' + value
#   elif re.search(r'^[1-9]\d', value[:2]):
#     value = '      ' + value
#   return value

# dict_month = {'Janeiro': '01', 'Fevereiro': '02', 'Março': '03', 'Abril': '04',
#               'Maio': '05', 'Junho': '06', 'Julho': '07', 'Agosto': '08',
#               'Setembro': '09', 'Outubro': '10', 'Novembro': '11', 'Dezembro': '12'}

In [118]:
# getting expense data
df_exp = pd.read_csv(r'/content/drive/MyDrive/Dashboard_data/final_data/df_exp.csv', sep=';', parse_dates=['Comp.pagto.'])

# getting budget settlement data from 2021
df_2021 = pd.read_excel(r'/content/drive/MyDrive/Dashboard_data/utils/consolidated_settlements_2021.xls', parse_dates=['Compet.Liq.'])
df_2021['Compet.Liq.'] = pd.to_datetime(df_2021['Compet.Liq.'], dayfirst=True)
df_2021 = df_2021.loc[
    (df_2021['Compet.Estorno'] == '21/12/2021') & (df_2021['Unid.Orçam.'] == 7002),
    ['Compet.Liq.', 'Unid.Orçam.', 'Proj/Ativ', 'Rubrica', 'Vinc.Orçam.', 'Val. Liquidado']
    ].copy().rename(columns={
        'Compet.Liq.': 'Comp.pagto.',
        'Unid.Orçam.': 'Unid. Orçam.',
        'Vinc.Orçam.': 'Vinc. Orçam.',
        'Val. Liquidado': 'Result. pago'
    })
df_2021['Elemento'] = df_2021['Rubrica'].astype(str).str[:6].astype(int)

# deleting specific 2021 data to replace it with budget settlement data
df_exp = df_exp[~((df_exp['Comp.pagto.'] == '2021-12-29') & (df_exp['Unid. Orçam.'] == 7002))]
df_exp = pd.concat([df_exp, df_2021], ignore_index=True)

# correcting data
df_exp.loc[
    (df_exp['Proj/Ativ'] == 2529) & (df_exp['Rubrica'] == 339036040000),
    ['Unid. Orçam.', 'Proj/Ativ', 'Elemento', 'Rubrica', 'Vinc. Orçam.']
] = 7003, 9075, 339039, 339039030000, 6050
df_exp.loc[
    (df_exp['Proj/Ativ'] == 9042) & (df_exp['Comp.pagto.'].dt.year > 2010),
    ['Unid. Orçam.', 'Proj/Ativ', 'Elemento', 'Rubrica']
] = 7002, 9076, 339086, 339086010000

# dropping 'Rubrica" column
df_exp.drop('Rubrica', axis=1, inplace=True)

In [119]:
# Selecting and clearing data from 7001
df_7001 = clean_7001(df_exp.loc[
    (df_exp['Unid. Orçam.'] == 7001) &
    (df_exp['Vinc. Orçam.'].isin([400, 1, 6050, 6069])) &
    (df_exp['Comp.pagto.'].dt.year > 2011) # period prior to GPREV removed
].copy())

# Selecting and clearing data from 7002
df_7002 = change_elements(df_exp.loc[
    (df_exp['Unid. Orçam.'] == 7002) &
    (~df_exp['Proj/Ativ'].isin([2737, 2739, 2741, 2743, 2745, 2746, 2748, 2750, 2753, 2755, 2757, 2759]))
].copy())
df_7002['Vinc. Orçam.'] = 6049

# Selecting and clearing data from 7003
df_7003 = change_elements(df_exp.loc[
    (df_exp['Unid. Orçam.'] == 7003) &
    (~df_exp['Proj/Ativ'].isin([2761, 2763, 2765, 2767, 2769, 2770, 2772, 2774, 2777, 2779, 2781, 2783]))
].copy())
df_7003['Vinc. Orçam.'] = 6050

In [120]:
# getting revenue data
df_rev = pd.read_csv(r'/content/drive/MyDrive/Dashboard_data/final_data/df_rev.csv', sep=';', parse_dates=['Data'])

# clearing revenue data
df_rev.drop(columns=['origem', 'tipo'], inplace=True)
df_rev = df_rev[(df_rev['vinculo'].isin([6050, 6069, 6049, 400])) & (df_rev['Data'].dt.year > 2017)]
df_rev.loc[df_rev['vinculo'] == 400, 'vinculo'] = 6049
df_rev.loc[
    (df_rev['vinculo'] == 6049) &
    (df_rev['nome_rubrica'] == 'Compensações Financ entre o Regime Geral e os RPPS'),
    'nome_rubrica'] = 'Compensações Financ entre o Regime Geral e os RPPS-Plano em Repartição'
df_rev.loc[
    (df_rev['vinculo'] == 6050) &
    (df_rev['nome_rubrica'] == 'Compensações Financ entre o Regime Geral e os RPPS'),
    'nome_rubrica'] = 'Comp. Financ. entre o Regime Geral e os RPPS - Plano em Capitalização'
df_rev.loc[
    (df_rev['vinculo'] == 6049) &
    (df_rev['nome_rubrica'] == 'Contr.do Servidor Civil Ativo - Cedido'),
    'nome_rubrica'] = 'Contr.do Servidor Civil Ativo - Cedido - Plano em Repartição'
df_rev.loc[
    (df_rev['vinculo'] == 6050) &
    (df_rev['nome_rubrica'] == 'Contr.do Servidor Civil Ativo - Cedido'),
    'nome_rubrica'] = 'Contr.do Servidor Civil Ativo - Cedido - Plano em Capitalização'
df_rev.loc[
    (df_rev['vinculo'] == 6049) &
    (df_rev['nome_rubrica'] == 'Contr.do Servidor Civil Ativo - Cedido - Multas e Juros'),
    'nome_rubrica'] = 'Contr.Serv.Civil Ativo - Cedido - Multas e Juros - Plano em Repartição'
df_rev.loc[
    (df_rev['vinculo'] == 6049) &
    (df_rev['nome_rubrica'] == 'Contr.do Servidor Civil Ativo - Cedido - Dív.At.- Multas e Juros'),
    'nome_rubrica'] = 'Contr.Serv.Ativo Cedido-Dív.Ativa-Multas e Juros - Plano em Repartição'
df_rev.loc[
    (df_rev['vinculo'] == 6049) &
    (df_rev['nome_rubrica'] == 'Contr.do Servidor Civil Ativo - Cedido - Dívida Ativa'),
    'nome_rubrica'] = 'Contr.Serv.Civil Ativo - Cedido - Dívida Ativa - Plano em Repartição'
df_rev.loc[
    (df_rev['vinculo'] == 6050) &
    (df_rev['nome_rubrica'] == 'Contr.do Servidor Civil Ativo - Cedido - Multas e Juros'),
    'nome_rubrica'] = 'Contr.Serv.Civil Ativo-Cedido-Multas e Juros - Plano em Capitalização'
df_rev.loc[
    (df_rev['vinculo'] == 6049) &
    (df_rev['nome_rubrica'] == 'Contr.Patronal - Serv. Afastados - Plano em Repartição'),
    'nome_rubrica'] = 'Contr.Patronal - Servidores Afastados - Plano em Repartição'
df_rev.loc[
    (df_rev['vinculo'] == 6049) &
    (df_rev['nome_rubrica'] == 'Contr.do Servidor Civil Ativo - Afastado'),
    'nome_rubrica'] = 'Contr.do Servidor Civil Ativo - Afastado - Plano em Repartição'
df_rev.loc[
    (df_rev['vinculo'] == 6050) &
    (df_rev['nome_rubrica'] == 'Contr.do Servidor Civil Ativo - Afastado'),
    'nome_rubrica'] = 'Contr.do Servidor Civil Ativo - Afastado - Plano em Capitalização'
df_rev.loc[
    (df_rev['vinculo'] == 6069) &
    (df_rev['nome_rubrica'] == 'Taxa de Administração - INTRA ORCAMENTÄRIA'),
    'nome_rubrica'] = 'Aluguéis Diversos - Taxa de Administração - INTRA ORCAMENTÄRIA'
df_rev.loc[
    (df_rev['vinculo'] == 6069) &
    (df_rev['nome_rubrica'] == 'Restituições de Servidores - RPPS'),
    'nome_rubrica'] = 'Restituições de Servidores - Taxa de Administração do RPPS'
df_rev.loc[
    (df_rev['vinculo'] == 6069) &
    (df_rev['nome_rubrica'].isin(['Restituições Diversas - RPPS', 'Outras Restituições Diversas'])),
    'nome_rubrica'] = 'Restituições Diversas - Taxa de Administração do RPPS'
df_rev.loc[
    (df_rev['vinculo'] == 6069) &
    (df_rev['nome_rubrica'].isin(['Restituições de Sobra de Adiantamento de Numerário - RPPS'])),
    'nome_rubrica'] = 'Sobra de Adiantamento de Numerário - Taxa de Administração do RPPS'
df_rev.loc[
    (df_rev['vinculo'] == 6049) &
    (df_rev['nome_rubrica'] == 'Restituições de Servidores - RPPS'),
    'nome_rubrica'] = 'Restituições de Servidores - RPPS - Plano em Repartição'
df_rev.loc[
    (df_rev['vinculo'] == 6049) &
    (df_rev['nome_rubrica'] == 'Restituições Diversas - RPPS'),
    'nome_rubrica'] = 'Restituições Diversas - RPPS - Plano em Repartição'
df_rev.loc[
    (df_rev['vinculo'] == 6050) &
    (df_rev['nome_rubrica'] == 'Restituições de Servidores - RPPS'),
    'nome_rubrica'] = 'Restituições de Servidores - RPPS - Plano em Capitalização'
df_rev.loc[
    (df_rev['vinculo'] == 6050) &
    (df_rev['nome_rubrica'].isin(['Restituições Diversas - RPPS', 'Restituições Diversas - Taxa de Administração do RPPS', 'Outras Restituições Diversas'])),
    'nome_rubrica'] = 'Restituições Diversas - RPPS - Plano em Capitalização'
df_rev = df_rev[~((df_rev['nome_rubrica'].str.lower().str.contains('patr')) & (df_rev['nome_rubrica'].str.lower().str.contains('inativo')))]
df_rev = df_rev[~df_rev['nome_rubrica'].str.contains('637|750|805|supl', case=False, regex=True)]

In [121]:
# preparing to predict
df_pred_exp = pd.concat([df_7001, df_7002, df_7003], ignore_index=True)
df_pred_exp['col'] = df_pred_exp['Unid. Orçam.'].astype(str) + '-' + df_pred_exp['Proj/Ativ'].astype(str) + '-' + df_pred_exp['Elemento'].astype(str) + '-' + df_pred_exp['Vinc. Orçam.'].astype(str)
df_pred_exp = df_pred_exp[~((df_pred_exp['col'].isin(mass_segregation_cols_exp)) & (df_pred_exp['Comp.pagto.'] < '2022-05'))]

df_pred_rev = df_rev.copy().rename(columns={'Data': 'Comp.pagto.', 'valor_arrecadado': 'Result. pago'})
df_pred_rev['col'] = df_pred_rev['nome_rubrica'] + '-' + df_pred_rev['vinculo'].astype(str)
df_pred_rev = df_pred_rev[~((df_pred_rev['col'].isin(mass_segregation_cols_rev)) & (df_pred_rev['Comp.pagto.'] < '2022-05'))]
df_pred_rev = df_pred_rev[~((df_pred_rev['col'].str.contains('pensionista|inativo|926/2021', case=False, regex=True)) & (df_pred_rev['Comp.pagto.'] < '2022-01'))]
df_pred_rev = df_pred_rev[~((df_pred_rev['col'].str.contains('contr', case=False, regex=False)) & (df_pred_rev['vinculo'] == 6069) & (df_pred_rev['Comp.pagto.'].dt.year < 2022))]

df_pred = pd.concat([df_pred_exp, df_pred_rev], ignore_index=True)
df_pred['Comp.pagto.'] = df_pred['Comp.pagto.'].dt.to_period('M')
df_pred = df_pred[['Comp.pagto.', 'col', 'Result. pago']].groupby(['Comp.pagto.', 'col'], as_index=False).sum()
df_pred = df_pred.pivot(index='Comp.pagto.', columns='col', values='Result. pago')
df_pred = df_pred.iloc[:-1]
df_pred.reset_index(inplace=True)
df_pred['T'] = np.arange(1, len(df_pred)+1)
df_pred = fill_zero(df_pred)

In [122]:
# prediction interval
start = df_pred['Comp.pagto.'].max()
current_year = start.year if start.month != 12 else start.year + 1
target = pd.Period(f'{current_year + 4}-12', freq='M')
month_diff = (target - start).n
last_T = df_pred['T'].max()
X_prev = pd.DataFrame({'T': [last_T + val for val in range(1, month_diff + 1)]})
years = list(range(current_year, current_year+5))
months = [start.to_timestamp() + pd.DateOffset(months=i) for i in range(1, month_diff + 1)]

In [128]:
def run_models(df: pd.DataFrame, df_pred: pd.DataFrame, years: list[int], X_prev: pd.DataFrame, start: pd.Period, months: list[pd.Timestamp], month_diff: int, ma: str = None) -> pd.DataFrame:
  df_models = pd.DataFrame({
      'Allocation': pd.Series(dtype='object'),
      'Model': pd.Series(dtype='object'),
      'R²': pd.Series(dtype='float'),
      'RMSE': pd.Series(dtype='float'),
      'MAE': pd.Series(dtype='float'),
      'Forecast': pd.Series(dtype='object'),
      **{year: pd.Series(dtype='float') for year in years}
  })

  def run_linear_models(X: np.ndarray | pd.DataFrame, y: np.ndarray | pd.Series, df_aux: pd.DataFrame, model: str, col: str) -> pd.DataFrame:
    # df_xy = pd.DataFrame({'X': X.flatten(), 'y': y.values})

    # df_train, df_test = time_based_train_test_split(df_xy)
    # if df_train is None:
    #     return df_aux

    lr_model = LinearRegression()
    # lr_model.fit(df_train[['X']], df_train['y'])

    # # Evaluate
    # if df_test is not None:
    #     y_pred_test = lr_model.predict(df_test[['X']])
    #     r2 = r2_score(df_test['y'], y_pred_test)
    #     RMSE = np.sqrt(mean_squared_error(df_test['y'], y_pred_test))
    #     MAE = mean_absolute_error(df_test['y'], y_pred_test)
    # else:
    #     y_pred_train = lr_model.predict(df_train[['X']])
    #     r2 = r2_score(df_train['y'], y_pred_train)
    #     RMSE = np.sqrt(mean_squared_error(df_train['y'], y_pred_train))
    #     MAE = mean_absolute_error(df_train['y'], y_pred_train)

    lr_model.fit(X, y)
    forecast = lr_model.predict(X)
    r2 = r2_score(y, forecast)
    RMSE = np.sqrt(mean_squared_error(y, forecast))
    MAE = mean_absolute_error(y, forecast)

    if model == 'lin-lin':
      y_pred = lr_model.predict(X_prev.values)
    elif model == 'log-log':
      y_pred = np.exp(lr_model.predict(np.log(X_prev).values))
    elif model == 'lin-log':
      y_pred = np.exp(lr_model.predict(X_prev.values))
    elif model == 'log-lin':
      y_pred = lr_model.predict(np.log(X_prev).values)
    else:
      raise ValueError(f"Unknown model type: {model}")

    forecast_df = pd.DataFrame({'date': months, 'y_pred': y_pred})
    forecast_df['year'] = forecast_df['date'].dt.year
    forecast_df['month'] = forecast_df['date'].dt.month

    if ma:
      december_forecasts = forecast_df[forecast_df['month'] == 12]
      if ma == '12':
        forecast_years = {year: val * 12 for year, val in zip(december_forecasts['year'], december_forecasts['y_pred'])}
      elif ma == '36':
        forecast_years = {}
        for i, (year, current_val) in enumerate(zip(december_forecasts['year'], december_forecasts['y_pred'])):
            current_total = current_val * 36
            if i == 0:
              past_2y_sum = df_pred.loc[df_pred['Comp.pagto.'].dt.year.isin([year-1, year-2]), col].sum()
              forecast_years[year] = current_total - past_2y_sum
            elif i == 1:
              past_2y_sum = forecast_years[year-1] + df_pred.loc[df_pred['Comp.pagto.'].dt.year == year-2, col].sum()
              forecast_years[year] = current_total - past_2y_sum
            else:
              past_2y_sum = forecast_years[year-1] + forecast_years[year-2]
              forecast_years[year] = current_total - past_2y_sum
    else:
      forecast_years = forecast_df.groupby('year')['y_pred'].sum().to_dict()
      if start.year in forecast_years:
        forecast_years[start.year] += df.loc[df['Comp.pagto.'].dt.year == start.year, col].sum()

    if check_negative_and_jump(forecast_years):
      return df_aux

    model_dict = {'Allocation': col, 'Model': model, 'R²': r2, 'RMSE': RMSE, 'MAE': MAE, 'Forecast': [forecast_df]}
    model_dict.update(forecast_years)
    df_aux = pd.concat([df_aux, pd.DataFrame(model_dict, index=[0])], ignore_index=True)
    return df_aux


  def run_prophet_model(df_aux: pd.DataFrame, col:str) -> pd.DataFrame:
    prophet_df = df[['Comp.pagto.', col]].copy().rename(columns={
        'Comp.pagto.': 'ds',
        col: 'y'
    }).dropna()

    # prophet_df = df[['Comp.pagto.', col]].rename(columns={
    #     'Comp.pagto.': 'ds',
    #     col: 'y'
    # }).dropna().reset_index(drop=True)

    if len(prophet_df) < 3:
      return df_aux

    prophet_df['ds'] = prophet_df['ds'].dt.to_timestamp()
    # prophet_df['floor'] = 0
    # prophet_df['cap'] = prophet_df['y'].max() * 1.1

    # df_train, df_test = time_based_train_test_split(prophet_df)
    # if df_train is None:
    #     return df_aux

    # len_test = len(df_test) if df_test is not None else 0

    model = build_prophet_model()

    # model = Prophet(yearly_seasonality=True)
    model.fit(prophet_df)
    # model.fit(df_train)

    # future = model.make_future_dataframe(periods=month_diff+len_test, freq='MS')
    future = model.make_future_dataframe(periods=month_diff, freq='MS')
    # future['floor'] = 0
    # future['cap'] = prophet_df['cap'].iloc[0]
    forecast = model.predict(future)

    # model.plot(forecast)

    # display(forecast)

    forecast_train = forecast[forecast['ds'].isin(prophet_df['ds'])]
    r2 = r2_score(prophet_df['y'], forecast_train['yhat'])
    RMSE = np.sqrt(mean_squared_error(prophet_df['y'], forecast_train['yhat']))
    MAE = mean_absolute_error(prophet_df['y'], forecast_train['yhat'])

    # if df_test is not None:
    #     forecast_test = forecast[['ds', 'yhat']].merge(df_test, on='ds', how='inner')
    #     r2 = r2_score(forecast_test['y'], forecast_test['yhat'])
    #     RMSE = np.sqrt(mean_squared_error(forecast_test['y'], forecast_test['yhat']))
    #     MAE = mean_absolute_error(forecast_test['y'], forecast_test['yhat'])
    # else:
    #     forecast_train = forecast[forecast['ds'].isin(prophet_df['ds'])]
    #     r2 = r2_score(prophet_df['y'], forecast_train['yhat'])
    #     RMSE = np.sqrt(mean_squared_error(prophet_df['y'], forecast_train['yhat']))
    #     MAE = mean_absolute_error(prophet_df['y'], forecast_train['yhat'])

    forecast_future = forecast[forecast['ds'] > prophet_df['ds'].max()]

    if ma:
      december_forecasts = forecast_future[forecast_future['ds'].dt.month == 12]
      if ma == '12':
        forecast_years = {year: val * 12 for year, val in zip(december_forecasts['ds'].dt.year, december_forecasts['yhat'])}
      elif ma == '36':
        forecast_years = {}
        for i, (year, current_val) in enumerate(zip(december_forecasts['ds'].dt.year, december_forecasts['yhat'])):
          current_total = current_val * 36
          if i == 0:
            past_2y_sum = df_pred.loc[df_pred['Comp.pagto.'].dt.year.isin([year-1, year-2]), col].sum()
            forecast_years[year] = current_total - past_2y_sum
          elif i == 1:
            past_2y_sum = forecast_years[year-1] + df_pred.loc[df_pred['Comp.pagto.'].dt.year == year-2, col].sum()
            forecast_years[year] = current_total - past_2y_sum
          else:
            past_2y_sum = forecast_years[year-1] + forecast_years[year-2]
            forecast_years[year] = current_total - past_2y_sum
    else:
      forecast_years = forecast_future.groupby(forecast_future['ds'].dt.year)['yhat'].sum().to_dict()
      if start.year in forecast_years:
        partial_actual_sum = prophet_df[prophet_df['ds'].dt.year == start.year]['y'].sum()
        forecast_years[start.year] += partial_actual_sum

    if check_negative_and_jump(forecast_years):
      return df_aux

    model_dict = {'Allocation': col, 'Model': 'prophet', 'R²': r2, 'RMSE': RMSE, 'MAE': MAE, 'Forecast': [forecast]}
    model_dict.update(forecast_years)
    df_aux = pd.concat([df_aux, pd.DataFrame(model_dict, index=[0])], ignore_index=True)
    return df_aux

  for col in df.columns.difference(['T', 'Comp.pagto.']):
    first_valid = df[col].first_valid_index()

    if first_valid is not None:
      X = df['T'][first_valid:].values.reshape(-1,1)
      y = df[col][first_valid:]

      if len(y) < 3 or y.eq(0).sum() / len(y) > 0.8:
        continue

      df_aux = pd.DataFrame()

      # lin-lin
      df_aux = run_linear_models(X, y, df_aux, 'lin-lin', col)

      # log-log
      if (y > 0).all():
        df_aux = run_linear_models(np.log(X), np.log(y), df_aux, 'log-log', col)

      # lin-log
      if (y > 0).all():
        df_aux = run_linear_models(X, np.log(y), df_aux, 'lin-log', col)

      # log-lin
      df_aux = run_linear_models(np.log(X), y, df_aux, 'log-lin', col)

      # prophet
      df_aux = run_prophet_model(df_aux, col)

      if df_aux.empty:
        continue

      df_models = pd.concat([df_models, df_aux], ignore_index=True)

  return df_models

In [82]:
36713358.136114374 - 27096163.0

9617195.136114374

In [88]:
10398611.54250858 + 14823375.076394208

25221986.618902788

In [80]:
df_pred.loc[df_pred['Comp.pagto.'].dt.year.isin([years[1]-1, years[1]-2]), '7001-4396-319011-6069'].sum()

np.float64(17281717.61)

In [115]:
17113229.804014094 + 13376495.102589011

30489724.906603105

In [129]:
models_debug = run_models(moving_averages(df_pred[
    ['Comp.pagto.', 'T', '7001-4396-319011-6069', '7002-2740-319001-6049', '7003-2771-319001-6050',
     'Contr. Patronal - Servidor Ativo - Plano em Repartição - Centralizada-6069', 'Contr. Patronal - Servidor Ativo - Plano em Repartição - Centralizada-6049',
     'Contr. Patronal - Servidor Ativo - Plano em Capitalização - DMAE-6050']
    ], 36), df_pred, years, X_prev, start, months, month_diff, '36')

In [130]:
ch_m = choose_models(models_debug)
ch_m

,Type,Model,R²,RMSE,MAE,Forecast,2025,2026,2027,2028,2029,r2_norm,rmse_norm,mae_norm,score
Allocation,,,,,,,,,,,,,,,
7001-4396-319011-6069,4,prophet,0.994646,5.009159e+03,3.617491e+03,ds trend yhat_lower ...,1.337650e+07,1.711323e+07,2.035341e+07,1.611628e+07,2.517606e+07,1.000000,0.808747,0.808587,0.904334
7002-2740-319001-6049,9,prophet,0.999180,1.357863e+05,1.056539e+05,ds trend yhat_lower ...,3.446404e+08,3.885768e+08,4.295392e+08,3.948751e+08,5.076534e+08,1.000000,0.839838,0.847759,0.921899
7003-2771-319001-6050,11,log-log,0.995861,7.929456e-02,6.528547e-02,date y_pred year month 0 20...,3.959270e+07,4.505351e+07,5.232761e+07,6.403141e+07,7.217119e+07,0.980903,1.000000,1.000000,0.990452
Contr. Patronal - Servidor Ativo - Plano em Capitalização - DMAE-6050,19,prophet,0.999995,1.637039e+02,1.163098e+02,ds trend yhat_lower ...,1.043188e+07,1.101175e+07,1.080066e+07,1.230632e+07,1.155794e+07,1.000000,0.987996,0.990343,0.994585
Contr. Patronal - Servidor Ativo - Plano em Repartição - Centralizada-6049,24,prophet,0.998457,1.555069e+04,1.230819e+04,ds trend yhat_lower ...,6.073485e+07,6.734918e+07,6.021747e+07,5.728369e+07,-1.043789e+06,1.000000,0.903145,0.904929,0.952019
Contr. Patronal - Servidor Ativo - Plano em Repartição - Centralizada-6069,25,lin-lin,1.000000,1.164153e-10,1.164153e-10,date y_pred year month 0 2...,7.500720e+06,6.797286e+06,6.609276e+06,5.614952e+06,4.911517e+06,1.000000,1.000000,1.000000,1.000000


In [133]:
25/16

1.5625

In [131]:
ch_m[years]

,2025,2026,2027,2028,2029
Allocation,,,,,
7001-4396-319011-6069,1.337650e+07,1.711323e+07,2.035341e+07,1.611628e+07,2.517606e+07
7002-2740-319001-6049,3.446404e+08,3.885768e+08,4.295392e+08,3.948751e+08,5.076534e+08
7003-2771-319001-6050,3.959270e+07,4.505351e+07,5.232761e+07,6.403141e+07,7.217119e+07
Contr. Patronal - Servidor Ativo - Plano em Capitalização - DMAE-6050,1.043188e+07,1.101175e+07,1.080066e+07,1.230632e+07,1.155794e+07
Contr. Patronal - Servidor Ativo - Plano em Repartição - Centralizada-6049,6.073485e+07,6.734918e+07,6.021747e+07,5.728369e+07,-1.043789e+06
Contr. Patronal - Servidor Ativo - Plano em Repartição - Centralizada-6069,7.500720e+06,6.797286e+06,6.609276e+06,5.614952e+06,4.911517e+06


In [134]:
models_debug

,Allocation,Model,R²,RMSE,MAE,Forecast,2025,2026,2027,2028,2029
0,7001-4396-319011-6069,lin-lin,0.894417,2.224553e+04,1.543077e+04,date y_pred year month 0 20...,9.617195e+06,1.383562e+07,1.482338e+07,1.039861e+07,1.461704e+07
1,7001-4396-319011-6069,log-log,0.882418,2.591831e-02,1.885987e-02,date y_pred year month 0 20...,8.694460e+06,1.351790e+07,1.448300e+07,9.115200e+06,1.392032e+07
2,7001-4396-319011-6069,lin-log,0.909635,2.272142e-02,1.634154e-02,date y_pred year month 0 20...,9.820394e+06,1.395726e+07,1.496710e+07,1.076817e+07,1.492822e+07
3,7001-4396-319011-6069,log-lin,0.853641,2.619121e+04,1.889879e+04,date y_pred year month 0 20...,8.586594e+06,1.346704e+07,1.442976e+07,8.952220e+06,1.381289e+07
4,7001-4396-319011-6069,prophet,0.994646,5.009159e+03,3.617491e+03,ds trend yhat_lower ...,1.337650e+07,1.711323e+07,2.035341e+07,1.611628e+07,2.517606e+07
5,7002-2740-319001-6049,lin-lin,0.991732,4.311262e+05,3.393635e+05,date y_pred year month 0 20...,3.768910e+08,3.844103e+08,3.938155e+08,4.288494e+08,4.363687e+08
6,7002-2740-319001-6049,log-log,0.989114,2.609029e-02,2.074964e-02,date y_pred year month 0 20...,3.371368e+08,3.710806e+08,3.794859e+08,3.738483e+08,4.069463e+08
7,7002-2740-319001-6049,lin-log,0.974515,3.992020e-02,3.500288e-02,date y_pred year month 0 20...,4.558289e+08,4.210263e+08,4.373735e+08,5.588313e+08,5.321015e+08
8,7002-2740-319001-6049,log-lin,0.968028,8.478043e+05,6.939904e+05,date y_pred year month 0 20...,2.955560e+08,3.577617e+08,3.656319e+08,3.179714e+08,3.789649e+08
9,7002-2740-319001-6049,prophet,0.999180,1.357863e+05,1.056539e+05,ds trend yhat_lower ...,3.446404e+08,3.885768e+08,4.295392e+08,3.948751e+08,5.076534e+08


In [11]:
# Run models separately
models_raw = run_models(df_pred, years, X_prev, start, months, month_diff)
models_ma12 = run_models(moving_averages(df_pred, 12), years, X_prev, start, months, month_diff, True)
models_ma36 = run_models(moving_averages(df_pred, 36), years, X_prev, start, months, month_diff, True)

# Combined results
combined = pd.concat([models_raw, models_ma12, models_ma36], keys=['raw', 'ma12', 'ma36'])

# chosen models
chosen_models = choose_models(combined)

In [12]:
# getting the history after cleaning
df_pred_year = (
    df_pred.drop(columns='T')
    .assign(year=df_pred['Comp.pagto.'].dt.year)
    .drop(columns='Comp.pagto.')
    .groupby('year')
    .sum()
    .T
)


# changing the name of the last column in the history, if necessary
# getting the last three columns of the history to calculate the average, if necessary
last_year_col = start.year
last_cols = df_pred_year.columns[-3:]
if start.month != 12:
  last_year_col = f"{str(start.year) + ' until ' + start.strftime('%B')}"
  df_pred_year.rename(columns={
      start.year: last_year_col
      }, inplace=True)
  last_cols = df_pred_year.columns.difference([last_year_col])[-3:]


# combining history with forecasting
df_past_future = pd.merge(df_pred_year, chosen_models[years], left_index=True, right_index=True, how='outer')


# calculating the average of the last three years for the columns not predicted by the models
df_past_future.loc[df_pred_year.index.difference(chosen_models.index), years] = df_pred_year.loc[df_pred_year.index.difference(chosen_models.index), last_cols].mean(axis=1).clip(lower=0)


# adjustments by rules
if start.month != 12:
  # define conditions and multipliers
  rules = [
      {
          'pattern': '319001|319003|319011|319013|319113|319092|319094|339047',
          'starts_with': '7',
          'multiplier': 12,
          'pattern_negate': True,
          'threshold_ratio': 1.03,
          'use_custom_ratio': False
      },
      {
          'pattern': '319001|319003|319011|319013|319113',
          'starts_with': None,
          'multiplier': 13,
          'pattern_negate': False,
          'threshold_ratio': 1.03,
          'use_custom_ratio': False
      },
      {
          'pattern': '319001|319003|319011',
          'starts_with': None,
          'multiplier': 13,
          'pattern_negate': False,
          'threshold_ratio': 1.3,
          'use_custom_ratio': True
      },
  ]

  for rule in rules:
      # pattern matching condition
      pattern_condition = ~df_past_future.index.str.contains(rule['pattern'], regex=True) if rule['pattern_negate'] else df_past_future.index.str.contains(rule['pattern'], regex=True)

      # optional: startswith condition
      startswith_condition = df_past_future.index.str.startswith(rule['starts_with']) if rule['starts_with'] else True

      # base projection
      base_proj = df_past_future[last_year_col] / start.month * rule['multiplier']

      # final condition
      final_condition = (df_past_future[current_year] > base_proj * rule['threshold_ratio']) if rule['use_custom_ratio'] else (base_proj > df_past_future[current_year] * rule['threshold_ratio'])

      # full condition
      condition = pattern_condition & startswith_condition & final_condition

      # ratio to scale other years
      ratio = base_proj * rule['threshold_ratio'] / df_past_future[current_year] if rule['use_custom_ratio'] else base_proj / df_past_future[current_year]

      # assign values
      df_past_future.loc[condition, years[0]] = base_proj * rule['threshold_ratio'] if rule['use_custom_ratio'] else base_proj
      for year in years[1:]:
          df_past_future.loc[condition, year] = df_past_future[year] * ratio


# adjusting the forecast for the current year, if the forecast value is lower than the value calculated in the year
df_past_future.loc[df_past_future[current_year] < df_past_future[last_year_col], current_year] = df_past_future[last_year_col]


# calculate social charges
charges = {
    'centralizada': '2736|2740|2742|2744|2747|2749|2751|2758|9074|9076',
    'demhab': '2752',
    'dmae': '2754',
    'dmlu': '2756',
    'fasc': '2738'
}

for k,v in charges.items():

  mask_k_charge = (
      df_past_future.index.str.contains(k, case=False, regex=False) &
      df_past_future.index.str.contains('encargo', case=False, regex=False)
  )

  sum_group = df_past_future.loc[
      df_past_future.index.str.contains(v, case=False, regex=True),
      years
  ].sum()

  if k != 'centralizada':
    sum_deduction = df_past_future.loc[
        df_past_future.index.str.contains(k, case=False, regex=False) &
        df_past_future.index.str.contains('6049', case=False, regex=False) &
        ~df_past_future.index.str.contains('encargo', case=False, regex=False),
        years
    ].sum()
  else:
    sum_deduction = df_past_future.loc[
        ~df_past_future.index.str.contains('demhab|dmae|dmlu|fasc|encargo', case=False, regex=True) &
        ~df_past_future.index.str.startswith('7') &
        df_past_future.index.str.contains('6049', case=False, regex=False),
        years
    ].sum()

  final_values = (sum_group - sum_deduction).clip(lower=0)

  df_past_future.loc[mask_k_charge, years] = [final_values.values] * mask_k_charge.sum()


# calculate PASEP treasury
df_past_future.loc['7001-9071-339047-1', years] = df_past_future.loc[(df_past_future.index.str.contains('encargo', case=False, regex=False)), years].sum() * 0.01

In [211]:
def choose_models(df: pd.DataFrame, rename_level: bool = True) -> pd.DataFrame:
  df_normalized = df.groupby('Allocation').apply(normalizes_calculates_score, include_groups=False)
  df_normalized.reset_index(inplace=True)
  chosen_models = (
      df_normalized
      .sort_values('score', ascending=False)
      .groupby('Allocation')
      .first()
  )
  if rename_level:
    chosen_models.rename(columns={'level_1': 'Type'}, inplace=True)
  chosen_models = chosen_models.loc[:, ~chosen_models.columns.map(lambda x: isinstance(x, str) and x.startswith('level'))]
  return chosen_models

In [203]:
def run_simple_linear_model(X: np.ndarray | pd.DataFrame, y: np.ndarray | pd.Series, model: str, col: str, df_models: pd.DataFrame) -> pd.DataFrame:
  lr_model = LinearRegression()
  lr_model.fit(X, y)
  forecast = lr_model.predict(X)
  r2 = r2_score(y, forecast)
  RMSE = np.sqrt(mean_squared_error(y, forecast))
  MAE = mean_absolute_error(y, forecast)

  years_array = np.array(years).reshape(-1, 1)

  if model == 'lin-lin':
    y_pred = lr_model.predict(years_array)
  elif model == 'log-log':
    y_pred = np.exp(lr_model.predict(np.log(years_array)))
  elif model == 'lin-log':
    y_pred = np.exp(lr_model.predict(years_array))
  elif model == 'log-lin':
    y_pred = lr_model.predict(np.log(years_array))
  else:
    raise ValueError(f"Unknown model type: {model}")

  forecast_years = {year: val for year, val in zip(years, y_pred)}
  model_dict = {'Allocation': col, 'Model': model, 'R²': r2, 'RMSE': RMSE, 'MAE': MAE }
  model_dict.update(forecast_years)

  df_models = pd.concat([df_models, pd.DataFrame(model_dict, index=[0])], ignore_index=True)

  return df_models

In [212]:
# getting the contribution wages
df_contrib = pd.read_excel(r'/content/drive/MyDrive/Dashboard_data/utils/contribution_wages-2018_2024.xlsx')

df_models = pd.DataFrame({
      'Allocation': pd.Series(dtype='object'),
      'Model': pd.Series(dtype='object'),
      'R²': pd.Series(dtype='float'),
      'RMSE': pd.Series(dtype='float'),
      'MAE': pd.Series(dtype='float'),
      **{year: pd.Series(dtype='float') for year in years}
  })

for col in df_contrib.drop(columns='Comp.pagto.'):
  X = df_contrib['Comp.pagto.'].values.reshape(-1,1)
  y = df_contrib[col]

  # lin-lin
  df_models = run_simple_linear_model(X, y, 'lin-lin', col, df_models)

  # log-log
  df_models = run_simple_linear_model(np.log(X), np.log(y), 'log-log', col, df_models)

  # lin-log
  df_models = run_simple_linear_model(X, np.log(y), 'lin-log', col, df_models)

  # log-lin
  df_models = run_simple_linear_model(np.log(X), y, 'log-lin', col, df_models)

# models_normalized = df_models.groupby('Allocation').apply(normalizes_calculates_score, include_groups=False)
# models_normalized.reset_index(inplace=True)
# models_normalized.drop(columns=['level_1'], inplace=True)
# chosen_models = (
#     models_normalized
#     .sort_values('score', ascending=False)
#     .groupby('Allocation')
#     .first()
# )

chosen_models = choose_models(df_models, rename_level=False)

In [213]:
chosen_models

,Model,R²,RMSE,MAE,2025,2026,2027,2028,2029,r2_norm,rmse_norm,mae_norm,score
Allocation,,,,,,,,,,,,,
Plano em Capitalização,lin-log,0.951976,0.034852,0.028648,9.555341e+08,1.032621e+09,1.115927e+09,1.205953e+09,1.303242e+09,1.0,1.0,1.0,1.0
Plano em Repartição,log-log,0.987124,0.017084,0.012908,4.122666e+08,3.826214e+08,3.551211e+08,3.296094e+08,3.059417e+08,1.0,1.0,1.0,1.0


In [181]:
# Filter the relevant data once
mask = (
    df_pred_rev['col'].str.contains(r'^Contr\.\s*Patronal.+6069$', regex=True) &
    (df_pred_rev['Comp.pagto.'] == start.to_timestamp())
)
df_contrib_by_co = df_pred_rev.loc[
      mask, ['Comp.pagto.', 'col', 'Result. pago']
    ].groupby(['Comp.pagto.', 'col'], as_index=False).sum()

# Compute proportions more efficiently
for tipo in ['Capitalização', 'Repartição']:
    mask_tipo = df_contrib_by_co['col'].str.contains(tipo)
    mask_chosen_models = chosen_models.index.str.contains(tipo)
    total = df_contrib_by_co.loc[mask_tipo, 'Result. pago'].sum()
    df_contrib_by_co.loc[mask_tipo, 'proportion'] = df_contrib_by_co.loc[mask_tipo, 'Result. pago'] / total

    # Get base value from previous year
    contrib_base = df_contrib.loc[
        df_contrib['Comp.pagto.'] == current_year - 1,
        df_contrib.columns.str.contains(tipo)
    ].squeeze() * 0.024

    df_contrib_by_co.loc[mask_tipo, current_year] = (
        contrib_base * df_contrib_by_co.loc[mask_tipo, 'proportion'].values
    )

    df_contrib_by_co.loc[mask_tipo, years[1:]] = (
        0.024 *
        df_contrib_by_co.loc[mask_tipo, 'proportion'].values[:, None] *
        chosen_models.loc[mask_chosen_models, years[:-1]].values
    )

df_contrib_by_co.set_index('col', inplace=True)
assert df_contrib_by_co.index.isin(df_past_future.index).all(), "Some contrib rows are not in past_future!"
df_past_future.loc[df_contrib_by_co.index, years] = df_contrib_by_co[years].values

df_past_future.loc['7003-9998-999999-6050', years] = df_past_future.loc[
    df_past_future.index.str.contains('6050') &
    ~df_past_future.index.str.startswith('7'), years].sum() - df_past_future.loc[
        df_past_future.index.str.contains('6050') &
        df_past_future.index.str.startswith('7'), years].sum()

df_past_future.loc['SuperávitOuDéfict-6069'] = df_past_future.loc[
    df_past_future.index.str.contains('6069') &
    ~df_past_future.index.str.startswith('7'), years].sum() - df_past_future.loc[
        df_past_future.index.str.contains('6069') &
        df_past_future.index.str.startswith('7'), years].sum()

In [184]:
df_past_future.loc['SuperávitOuDéfict-6069']

,SuperávitOuDéfict-6069
2011,NaN
2012,NaN
2013,NaN
2014,NaN
2015,NaN
2016,NaN
2017,NaN
2018,NaN
2019,NaN
2020,NaN


In [182]:
df_past_future.loc[df_past_future.index.str.contains('6069') & ~df_past_future.index.str.startswith('7'), years].sum()

,0
2025,3.961304e+07
2026,3.833706e+07
2027,3.788935e+07
2028,3.969860e+07
2029,3.903391e+07


In [183]:
df_past_future.loc[df_past_future.index.str.contains('6069') & df_past_future.index.str.startswith('7'), years].sum()

,0
2025,2.747189e+07
2026,2.926160e+07
2027,3.210200e+07
2028,3.308743e+07
2029,3.696534e+07


In [ ]:
rubricas = df_pred_rev.columns.get_level_values('nome_rubrica').str.lower()
mask = rubricas.str.contains('682')# & ~rubricas.str.contains('outr') & ~rubricas.str.contains('afast')
df_pred_rev.loc[:, df_pred_rev.columns[mask]]

vinculo                                                                        6050  \
nome_rubrica Parcel. Déb. - Patr. - Termo 00682/18 - LEI 12.371/2018 - Centralizada   
Data                                                                                  
2018-01                                                     NaN                       
2018-02                                                     NaN                       
2018-03                                                     NaN                       
2018-04                                                     NaN                       
2018-05                                                     NaN                       
...                                                         ...                       
2024-11                                                     NaN                       
2024-12                                              1017166.12                       
2025-01                                               518527.81                       
2025-02                                               519359.37                       
2025-03                                               528764.29                       

vinculo                                                                        \
nome_rubrica Parcel. Déb. - Patr. - Termo 00682/18 - LEI 12.371/2018 - DEMHAB   
Data                                                                            
2018-01                                                     NaN                 
2018-02                                                     NaN                 
2018-03                                                     NaN                 
2018-04                                                     NaN                 
2018-05                                                     NaN                 
...                                                         ...                 
2024-11                                                  449.98                 
2024-12                                                     NaN                 
2025-01                                                  460.79                 
2025-02                                                  461.53                 
2025-03                                                  469.89                 

vinculo                                                                      \
nome_rubrica Parcel. Déb. - Patr. - Termo 00682/18 - LEI 12.371/2018 - DMLU   
Data                                                                          
2018-01                                                     NaN               
2018-02                                                     NaN               
2018-03                                                     NaN               
2018-04                                                     NaN               
2018-05                                                     NaN               
...                                                         ...               
2024-11                                                  294.68               
2024-12                                                  751.19               
2025-01                                                  301.76               
2025-02                                                  302.24               
2025-03                                                  307.71               

vinculo                                                                      \
nome_rubrica Parcel. Déb. - Patr. - Termo 00682/18 - LEI 12.371/2018 - FASC   
Data                                                                          
2018-01                                                     NaN               
2018-02                                                     NaN               
2018-03                                                     NaN               
2018-04                                                     NaN  